### Carregamento de dados

In [ ]:
import numpy as np
import pandas as pd

notas_so1 = pd.read_excel("modulo1.4-dados.xlsx")
notas_so1

,ID,Nota 1,Nota 2,Nota 3,Nota 4
0,1,8.3,8.0,9.3,8.0
1,2,7.0,7.7,4.6,8.7
2,3,7.8,3.2,7.7,5.1
3,4,7.3,9.3,9.8,4.8
4,5,5.8,9.8,10.0,6.8
5,6,9.6,9.8,10.0,9.0
6,7,9.0,8.5,8.5,8.0
7,8,4.0,7.5,5.3,2.0
8,9,10.0,9.9,9.6,9.1
9,10,8.3,8.7,8.0,6.7


In [ ]:
# comando info para obter informações sobre tipos
# também mostra a presença de dados nulos
notas_so1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0    ID     21 non-null     int64  
 1   Nota 1  21 non-null     float64
 2   Nota 2  21 non-null     float64
 3   Nota 3  21 non-null     float64
 4   Nota 4  21 non-null     float64
dtypes: float64(4), int64(1)
memory usage: 972.0 bytes


In [ ]:
# gera estatísticas descritivas de um DataFrame ou Series
notas_so1.describe()

# count : nº de valores não nulos (não-NA)(non-null)
# mean  : média 
# std   : desvio padrão (medida de dispersão dos dados em torno da média)
# min   : valor mínimo presente na coluna
# 25%   : primeiro quartil (25% dos dados estão abaixo desse valor)
# 50%   : mediana (segundo quartil, 50% dos dados estão abaixo desse valor)
# 75%   : terceiro quartil (75% dos dados estão abaixo desse valor)
# max   : valor máximo presente na coluna

,ID,Nota 1,Nota 2,Nota 3,Nota 4
count,21.000000,21.000000,21.000000,21.000000,21.000000
mean,11.000000,7.595238,8.438095,7.771429,6.742857
std,6.204837,2.408003,2.020266,2.242129,2.510492
min,1.000000,0.000000,2.500000,1.000000,0.000000
25%,6.000000,7.000000,8.000000,7.000000,5.100000
50%,11.000000,8.300000,9.000000,8.100000,7.200000
75%,16.000000,9.300000,9.800000,9.600000,8.700000
max,21.000000,10.000000,10.000000,10.000000,9.800000


In [ ]:
# calcular media de cada unidade
notas_so1.mean(numeric_only=True)

 ID       11.000000
Nota 1     7.595238
Nota 2     8.438095
Nota 3     7.771429
Nota 4     6.742857
dtype: float64

### Supunhamos que eu tivesse força de vontade para digitar as notas de outro período:

In [ ]:
# carregando dados
notas_so2 = pd.read_excel("pseudonotas2.xlsx")

### Conectando dataframes

In [ ]:
# definindo coluna período para diferenciar após mescla
notas_so1["periodo"] = "2025.1"
notas_so2["periodo"] = "2025.2"

# enfim, a mescla
notas_final = pd.concat([notas_so1, notas_so2])

### Filtrando Dados (Forma 1 - Rústica)

In [ ]:
# media unidade 1 periodo .1
notas_final[notas_final["periodo"] == "2025.1"]["Nota 1"].mean()

# media unidade 1 periodo .2
notas_final[notas_final["periodo"] == "2025.2"]["Nota 1"].mean()

# fração de alunos com nota acima da média na unidade 1 periodo .1
len(notas_final[(notas_final["periodo"] == "2025.1") & (notas_final["Nota 1"] >= 7)].index) / len(notas_final[(notas_final["periodo"] == "2025.1")].index)
# pegou as notas >= 7 e dividiu pela quantidade de notas

### Filtrando Dados (Forma 2 - GroupBy)

In [ ]:
# media todas as unidades agrupadas por periodo
media_por_periodo = notas_final.groupby(["periodo"]).mean()
media_por_periodo

# fração de alunos com nota acima da média na unidade 1 para cada periodo
def prop_aprovados(df, col="Nota 1", nota_aprovacao=7):
    return len(df[df["Nota 1"] >= nota_aprovacao].index) / len(df.index)
    
notas_final.groupby(["periodo"]).apply(prop_aprovados)

### tidy data
forma de organizar dados que facilita a leitura de dados por parte das ferramentas de análise

#### Existem 3 regras para uma tabela ser tidy:
* Cada variável deve ter sua própria coluna.
* Cada observação deve ter sua própria linha.
* Cada valor deve ter sua própria célula.

### Usando o Pandas para transformar os dados no formato tidy, usando funções de reshaping e pivot tables
https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html

 Primeiro para as notas de Sistemas Operacionais, vamos adicionar uma coluna com o nome da disciplina e transformar as colunas nota_1, nota_2, nota_3 e nota_4 em duas colunas: unidade e nota. Ou seja, um aluno da turma ao invés de ter as 4 notas em apenas uma linha, ele terá as notas distribuídas em 4 linhas, uma para cada unidade. Para isto, vamos usar a função do Pandas wide_to_long:

In [ ]:
# isso aqui seria supondo que tivéssemos adicionado notas de outras disciplinas no nosso arquivo
so["disciplina"] = "Sistemas Operacionais"
so_tidy = pd.wide_to_long(so, stubnames='nota', i=['id', 'periodo', 'disciplina'],
                          j='unidade', sep="_")
so_tidy

# O parâmetro stubname='nota' indica que ele vai transformar as colunas que começam com a palavra “nota”. 
# O parâmetro i=['id', 'periodo', 'disciplina'] indica as colunas que identificarão unicamente cada linha de observação (serão os índices).
# E o parâmetro j='unidade' indica o nome da nova coluna que será criada, onde os valores serão os números encontrados nas colunas nota_X, considerando que sep="_" e os nomes das colunas têm o formato nota_unidade.

Fazendo isso agora para as outras disciplinas e concatenando tudo em um único DataFrame. Vamos também renomear o index id para id_aluno para melhor compreensão dos dados. O parâmetro inplace=True faz com que o DataFrame notas seja alterado ao invés de retornar um novo DataFrame com a alteração realizada.

In [ ]:
ads_tidy = pd.wide_to_long(ads_20171, stubnames='nota', i=['id', 'periodo', 'disciplina'], j='unidade', sep="_")
sd_tidy = pd.wide_to_long(sd_20172, stubnames='nota', i=['id', 'periodo', 'disciplina'], j='unidade', sep="_")

notas = pd.concat([so_tidy, ads_tidy, sd_tidy])
notas.index.set_names({'id': 'id_aluno'}, inplace=True)
        
notas

Como calcular agora a nota média para cada disciplina, período e unidade? Ficou bem mais fácil.

### Média da turma por unidade

In [ ]:
notas.groupby(["disciplina", "periodo", "unidade"]).mean()

Em resumo, unidade é como uma lista que armazena as 3-4 notas e periodo armazena 2 periodos, nesse fim, irá separar por disciplina, período, unidade e média de cada unidade.
a disciplina tem os períodos(2), os períodos tem as unidades(3-4) e cada unidade tem sua média (única)


### Média de cada aluno em cada turma

In [ ]:
media_alunos = notas.groupby(["disciplina", "periodo", "id_aluno"]).mean()
media_alunos

disciplinas tem períodos, que por sua vez tem alunos, cada um com sua nota única

### Média geral de cada turma

In [ ]:
media_turma = media_alunos.groupby(["disciplina", "periodo"]).mean()
media_turma

cada disciplina tem seu(s) período(s) (1-2) que, por sua vez, tem suas médias

### Outras estatísticas?
E se eu quiser calcular outras estatísticas como a quantidade de alunos, nota mínima, mediana, média e máxima? Podemos usar a função agg para aplicar várias funções de agregação nos dados.

In [ ]:
def prop_aprovados(medias, min_media_aprovacao=5):
    return medias[medias >= min_media_aprovacao].count() / medias.count()

(
    media_alunos
    .groupby(["disciplina", "periodo"])
    .agg(['count', 'min', 'median', 'mean', 'max', prop_aprovados])
)

Você também pode aplicar uma função própria, por exemplo a que calcula a proporção de alunos aprovados em cada disciplina, que conta as linhas apenas de alunos com nota maior ou igual à 5 e divide pela quantidade total de alunos dentro do grupo:

In [ ]:
def prop_aprovados(medias, min_media_aprovacao=5):
    return medias[medias >= min_media_aprovacao].count() / medias.count()

(
    media_alunos
    .groupby(["disciplina", "periodo"])
    .agg([prop_aprovados])
"APROVADO" if media_alunos["nota"] >= 5 else "REPROVADO")

### Adicionando nova coluna aplicando função por linha
Vamos adicionar uma coluna na tabela indicando se o aluno foi aprovado por média ou não. Para isso, vamos criar a nova função e aplicá-la para cada linha do dataframe com a função apply

In [ ]:
def calcula_resultado(media):
    return 'APROVADO' if media >= 5 else 'REPROVADO'

media_alunos["resultado"] = media_alunos["nota"].apply(calcula_resultado)
media_alunos

### Ordenando as médias dos alunos da maior para a menor
A função sort_values ordena o dataframe com base nos valores de uma ou mais colunas. O parâmetro ascending=False é usado para indicar que a ordenação não será feita na ordem padrão ascendente (ou seja, será na ordem descendente do maior para o menor valor)

In [ ]:
media_alunos.sort_values('nota', ascending=False)

### Como pegar as 3 maiores notas de cada turma?
Pega o dataframe com a média dos alunos, ordena pela média em ordem decrescente, agrupa por turma (disciplina e período) e pega as 3 primeiras linhas de cada grupo


In [ ]:
(
    media_alunos
    .sort_values('nota', ascending=False)
    .groupby(['disciplina', 'periodo'])
    .head(3)
)

### Slicing
Outra utilidade do Pandas é pegar apenas “fatias” (slices) dos dados, seja filtrando linhas ou colunas. O pandas oferece a função .loc para fazer o slicing a partir dos índices e a função .iloc para filtrar com base no número da linha.

Por exemplo, para pegar apenas as linhas que tem “Sistemas Operacionais” como primeiro índice (disciplina), podemos usar:

In [ ]:
media_alunos.loc["Sistemas Distribuidos"]

#E se quisermos pegar apenas as linhas 5 a 10 do DataFrame podemos usar:
media_alunos.iloc[5:10] # 6, 7, 8, 9, 10

Podemos também fazer um slice de linha e coluna ao mesmo tempo. Por exemplo, para filtrar as linhas com índices de 0 a 4 e a coluna nota_3, podemos fazer:

In [ ]:
so.loc[0:4, "nota_3"]

# SAÍDA:
# 0     9.3
# 1     4.6
# 2     7.7
# 3     9.8
# 4    10.0
# Name: nota_3, dtype: float64

Ou para pegar o valor que está na linha 10 e coluna 3 (lembrando que Python indexa a partir do 0):

In [ ]:
so.iloc[10, 3]

## Saída de dados
Também podemos exportar um dataframe pra um arquivo. Por exemplo, para exportar a tabela de média dos alunos para um arquivo podemos rodar:

In [ ]:
media_alunos.to_csv("../dados/notas_medias_alunos.csv")